# Global Variable (Edit Before you Run on your own)

In [1]:
EPOCHS_SIZE = 30 #Adjusted for trial first
BATCH_SIZE = 128
MODEL_SAVE_NAME = "digit_symbol_model_v2_with_30epochs" #change this so that u dont overwrite saved model
LOADED_MODEL_NAME = "" #Edit this one below

## Pre requisites that you need to install before use

#Just Run Once
!pip install tensorflow
!pip install scikit-learn
!pip install keras
!pip install keras-tuner
!pip install matplotlib
!pip install opencv-python
!pip install scipy

## 1. Imports

In [2]:
import os
import glob
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import seaborn as sns
from keras import metrics

Using TensorFlow backend


## 2. Loading The Picture

In [3]:
def pre_processing_from_dir(dataset_dir, class_labels_dict, training=False):
    # Initialize lists to store images and labels
    images = []
    labels = []
    class_labels = []

    # Get a list of all subdirectories (each subdirectory represents a class)
    class_directories = os.listdir(dataset_dir)

    # Iterate through each subdirectory (class directory)
    for class_directory in class_directories:
        class_label = class_directory  # Use the directory name as the class label
        
        class_labels.append(class_label)
        class_path = os.path.join(dataset_dir, class_directory)

        # Get a list of image files in the class directory
        image_files = glob.glob(os.path.join(class_path, "*.jpg"))  # You may need to adjust the file extension

        # print(image_files)

        # Iterate through image files in the class directory
        for image_file in image_files:
            # Load and preprocess the image
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (28, 28))
            image = image / 255.0  # Normalize pixel values

            # plt.imshow(image, cmap=plt.cm.binary)

            # Append the preprocessed image and its label to the lists
            images.append(image)
            labels.append(class_label)

    if training:

        data = list(zip(images, labels))

        # Shuffle the combined data
        np.random.shuffle(data)

        # shuffle the training images
        shuffled_images, shuffled_labels = zip(*data)

        images = np.array(shuffled_images)

        label_encoder = LabelEncoder()

        # Encode class labels using LabelEncoder
        labels = label_encoder.fit_transform(shuffled_labels)

        for i in range(len(class_labels)):
            class_labels_dict[class_labels[i]] = i

        labels = np.array(labels, dtype="int64")

        # comment the below 2 lines if doing label-encoding
        # One-hot encode labels (need to do one code in order to fit into the model)
        num_classes = len(class_labels)
        labels = to_categorical(labels, num_classes=num_classes)

        return images, labels, class_labels, class_labels_dict

    else:
       
        # Convert lists to NumPy arrays
        images = np.array(images)

        # label-encoding done on test data should correspond to the ones in training data
        # this is to account for times when test data is lesser than training data

        for i in range(len(labels)):
            labels[i] = class_labels_dict[labels[i]]

        labels = np.array(labels, dtype="int64")

        # comment the below 2 lines if doing label-encoding
        # One-hot encode labels (need to do one code in order to fit into the model)
        num_classes = len(class_labels_dict)
        labels = to_categorical(labels, num_classes=num_classes)

        return images, labels, class_labels

In [5]:
images, labels, training_class_labels, class_labels_dict = pre_processing_from_dir("final_82/train_images", {}, True)

### 2.2 Exploratory Data Analysis

### Data Augmentation on Training Data 
- random rotation
- random noise

In [6]:
def data_augmentation(image):

    ##############################################################
    # Rotating images to mimic slanted handwriting

    # Convert the image to a NumPy array (assuming it's in the range [0, 1])
    image = (image * 255).astype(np.uint8)

    # Calculate the image center
    center = tuple(np.array(image.shape[1::-1]) / 2)

    rotation_angle = random.uniform(-30, 30)

    # Create a rotation matrix and apply the rotation
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))

    # Convert back to the range [0, 1]
    rotated_image = rotated_image.astype(np.float32) / 255.0

    ##############################################################
    # Adding random noise to mimic low quality images

    max_noise_level = random.uniform(0, 0.1)
    noise = tf.random.normal(shape=tf.shape(rotated_image), stddev=max_noise_level)
    
    return tf.clip_by_value(rotated_image + noise, 0.0, 1.0)


## 3. Defining The Model

In [7]:
from keras import callbacks

#Define callback functions. Change model_name, patience, as needed.
# keras_callbacks   = [
#       callbacks.EarlyStopping(monitor='val_loss', patience=15, mode='min', min_delta=0.0001),
#       callbacks.ModelCheckpoint('model_name', monitor='val_loss', save_best_only=True, mode='min')

def math_model(images, labels, num_classes, model_name):

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    # perform data augmentation on X_train
    X_train = np.array([data_augmentation(image) for image in X_train])

    # Define your CNN model for multi-class classification

    input_shape = (28,28,1) # decision point: what size are our images fixed at
    layer1_size = 32 # number of filters in the convolutional layer
    layer2_size = 64
    layer3_size = 128
    layer_shape = (3,3) # size of the filter

    pool_shape = (2,2) # size of the pooling laye
    fully_connected_layer_size = 128 # number of neurons in the fully connected layer


    model = Sequential([
        layers.Conv2D(layer1_size, layer_shape, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_shape),
        layers.Conv2D(layer2_size, layer_shape, activation='relu'),
        layers.MaxPooling2D(pool_shape),
        layers.Conv2D(layer3_size, layer_shape, activation='relu'),
        layers.Flatten(),
        layers.Dense(fully_connected_layer_size, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model_metrics = ['accuracy', metrics.Recall(name = "Recall"), metrics.Precision(name = "Precision")]

    # Compile the model
    # for one-hot encoding
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=model_metrics)

    # uncomment this if using label-encoding, & comment the one above
    # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=model_metrics)

    # Fit model. (Batch size either 32, 64, 128. 1000 epochs as we expect training to stop before that.
    history = model.fit(X_train, y_train, batch_size=BATCH_SIZE,
                    epochs=EPOCHS_SIZE, validation_data=(X_test, y_test))

    # Save the trained model for later use
    model.save(f"{model_name}.keras")

    # not a must to return history here but it's to see whether model is overfitting or underfitting after training
    # can remove history once we confirmed model is good
    return f"{model_name}.keras", history, X_test, y_test

### Define the Prediction Function

In [8]:
def math_reports(model, X_test, y_test, test_class_labels, train_class_labels):

    # Load the saved model
    loaded_model = load_model(model)

    predicted_y = loaded_model.predict(X_test)

    # Convert one-hot encoded labels back to integer labels
    y_test_labels = y_test

    # comment this if label-encoding was used
    y_test_labels = np.argmax(y_test_labels, axis=1)

    predicted_labels = np.argmax(predicted_y, axis=1)

    confusion = confusion_matrix(y_test_labels, predicted_labels)

    # print("Confusion Matrix")
    # print(confusion)
    # print()

    cf_report = classification_report(y_test_labels, predicted_labels, labels=np.unique(y_test_labels), target_names=test_class_labels)

    # print("Classification Report")
    # print(cf_report)


    # print("Confusion Matrix Report")
    # Initialize dictionaries to store correct and total counts for each class
    correct_instances_per_class = {}
    total_instances_per_class = {}
    report = ""
    predicted_report = ""

    # Iterate through predictions and true labels to calculate correct and total instances
    for i in range(predicted_labels.size):
        predicted = train_class_labels[predicted_labels[i]]
        test_label = train_class_labels[y_test_labels[i]]

        result = "wrong"

        if (predicted == test_label):
            result = "correct"

        predicted_report += f"Predicted: {predicted}, Actual: {test_label}, Result: {result}\n"

        if test_label not in correct_instances_per_class:
            correct_instances_per_class[test_label] = 0
            total_instances_per_class[test_label] = 0

        total_instances_per_class[test_label] += 1

        if predicted == test_label:
            correct_instances_per_class[test_label] += 1

    # print(predicted_report)

    import operator

    sorted_correct = dict(sorted(correct_instances_per_class.items(), key=operator.itemgetter(0)))

    # Print the summary of correct/total for each class
    for label in sorted_correct:
        correct_count = sorted_correct[label]
        total_count = total_instances_per_class[label]
        report += f"Class {label}: Correct {correct_count}/{total_count} | Wrong: {total_count - correct_count}\n"

    # print(report)

    return confusion, cf_report, report, predicted_report, predicted_labels

## 4. Training Data

In [11]:
model, history, X_test, y_test = math_model(images, labels, len(training_class_labels), MODEL_SAVE_NAME)

Epoch 1/30
1121/1121 [==============================] - 52s 46ms/step - loss: 0.6755 - accuracy: 0.7920 - Recall: 0.7263 - Precision: 0.9003 - val_loss: 0.2443 - val_accuracy: 0.9259 - val_Recall: 0.9175 - val_Precision: 0.9370
Epoch 2/30
1121/1121 [==============================] - 51s 46ms/step - loss: 0.2469 - accuracy: 0.9242 - Recall: 0.9116 - Precision: 0.9390 - val_loss: 0.1511 - val_accuracy: 0.9558 - val_Recall: 0.9532 - val_Precision: 0.9595
Epoch 3/30
1121/1121 [==============================] - 50s 45ms/step - loss: 0.1815 - accuracy: 0.9442 - Recall: 0.9372 - Precision: 0.9519 - val_loss: 0.1240 - val_accuracy: 0.9631 - val_Recall: 0.9616 - val_Precision: 0.9654
Epoch 4/30
1121/1121 [==============================] - 50s 45ms/step - loss: 0.1481 - accuracy: 0.9536 - Recall: 0.9489 - Precision: 0.9592 - val_loss: 0.1357 - val_accuracy: 0.9569 - val_Recall: 0.9548 - val_Precision: 0.9600
Epoch 5/30
1121/1121 [==============================] - 50s 45ms/step - loss: 0.1291 - a

### Check for under/overfitting & deciding on epoch

In [ ]:
# LOADED_MODEL_NAME = "" #change if you want load another model
LOADED_MODEL_NAME = model

In [ ]:
# Load the saved model
loaded_model = load_model(LOADED_MODEL_NAME)

loss, accuracy = loaded_model.evaluate(X_test, y_test)
print(accuracy)
print(loss)

# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
confusion, cf_report, report, predicted_report, predicted_labels = math_reports(model, X_test, y_test, training_class_labels, training_class_labels)

In [ ]:
print("Confusion Matrix")
print(confusion)

In [ ]:
print("Classification Report")
print(cf_report)

In [ ]:
print("Confusion Matrix Report")
print(report)

In [ ]:
print("Predicted vs Actual")
print(predicted_report)

## 5 Test Model with Unseen Test Data

In [10]:
#load test_images
images_test, labels_test, test_class_labels = pre_processing_from_dir("dataset/test", class_labels_dict)

KeyError: '!'

In [ ]:
#Predict using loaded model
confusion_test, cf_report_test, report_test, predicted_report_test, predicted_labels_test = math_reports(LOADED_MODEL_NAME, images_test, labels_test, test_class_labels, training_class_labels)

In [ ]:
print("Classification Report Test")
print(cf_report_test)

In [ ]:
print("Confusion Matrix Test")
print(confusion_test)

In [ ]:
# to better visualize confusion matrix

# Replace this with your class labels
class_labels = [key for key, value in class_labels_dict.items() if value in np.unique(predicted_labels_test)]

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.set(font_scale=1.2)
sns.heatmap(confusion_test, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

# Set labels and title
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix Heatmap')

# Show the plot
plt.show()

In [ ]:
print("Confusion Matrix Report")
print(report_test)

In [ ]:
print("Predicted vs Actual Test")
print(predicted_report_test)

## running model with unseen data

In [12]:
def pre_processing_from_test(dataset_dir):
    # Initialize lists to store images and labels
    images = []
    labels = []
    class_labels = []

    # Get a list of all subdirectories (each subdirectory represents a class)
    class_directories = os.listdir(dataset_dir)

    # Iterate through each subdirectory (class directory)
    for class_directory in class_directories:
        class_label = class_directory  # Use the directory name as the class label
        
        class_labels.append(class_label)
        class_path = os.path.join(dataset_dir, class_directory)

        # Get a list of image files in the class directory
        image_files = glob.glob(os.path.join(class_path, "*.png"))  # You may need to adjust the file extension

        # print(image_files)

        # Iterate through image files in the class directory
        for image_file in image_files:
            # Load and preprocess the image
            image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (28, 28))
            image = image / 255.0  # Normalize pixel values

            # plt.imshow(image, cmap=plt.cm.binary)

            # Append the preprocessed image and its label to the lists
            images.append(image)
            labels.append(class_label)

    return np.array(images), class_labels

In [13]:
ez_test, ez_labels = pre_processing_from_test("final_82/ez")

In [18]:
training_class_labels = ['(',')','+','-','0','1','2','3','4','5','6','7','8','9','=','div','times']

# Load the saved model
model = "digit_symbol_model_v2_with_30epochs.keras"

loaded_model = tf.keras.saving.load_model(model)
#loaded_model = tf.keras.models.load_model(model)

predicted_y = loaded_model.predict(ez_test)

predicted_labels = np.argmax(predicted_y, axis=1)

print("predicted_labels: ", predicted_labels)

predicted_report = ""

# Iterate through predictions and true labels to calculate correct and total instances
for i in range(predicted_labels.size):
    predicted = training_class_labels[predicted_labels[i]]
    predicted_report += predicted + " "

print("predicted output: ", predicted_report)

1/1 [==============================] - 0s 60ms/step
predicted_labels:  [ 4 10 12]
predicted output:  0 6 8 


# Evaluate with ground Truth


In [35]:
import pandas as pd

# Read the CSV file
filtered_data = pd.read_excel("ten_paths.xlsx")

filtered_data

,path,gt
0,20_em_41,9/5
1,RIT_2014_29,- 7
2,RIT_2014_70,1 + 1 + 1 + 1 + 1 = 5
3,35_em_15,1 \times 1 + 1 \times 2 + 2 \times 2
4,RIT_2014_284,- 39
5,RIT_2014_299,897
6,515_em_351,1 = 1(1)(1)
7,RIT_2014_204,47474 + 5272 = 52746
8,18_em_21,1011\ 1110\ 1110\ 0101_2


### Convert the dataframe into dictionary

In [24]:
filtered_data_dictionary = filtered_data.set_index('path')['gt'].to_dict()
print(filtered_data_dictionary)

{'ez': '1 + 2 ', 'rit_4235_3': ' \\frac {5} {6} ', 'rit_4240_0': ' 57753336 ', 'rit_4210_4': ' \\frac {9} {7} ', 'rit_4225_2': ' 8 \\sqrt {3} ', 'rit_4225_3': ' \\frac {56} {8} ', 'rit_4295_0': ' 523 + 487 ', 'rit_4250_3': ' \\sqrt {18} ', 'ritm_422_0': ' + 7 \\times 9 ^ {5} + 7 \\times 9 ^ {4} + 7 \\times 9 ^ {3} + 7 \\times 9 ^ {2} + 7 \\times 9 ', 'rit_42100_2': ' \\frac {\\sqrt {4}} {2} ', 'rit_420_3': ' 9 ^ {9 ^ {9 ^ {9 ^ {9 ^ {9}}}}} ', 'rit_4230_0': ' \\frac {7} {6} ', 'rit_4260_4': ' \\frac {1} {8} ', 'rit_4230_2': ' \\frac {3} {7} ', 'rit_4250_2': ' \\sqrt {28} ', 'rit_4280_0': ' \\frac {91} {48} ', 'rit_4265_0': ' 4 ^ {4 ^ {4}} + 3 ', 'rit_4290_4': ' \\sqrt {3 + \\sqrt {2}} ', 'rit_42190_4': ' \\sqrt {- 1} \\times \\sqrt {- 1} ', 'rit_4245_3': ' 7 \\sqrt {7} - 3 \\sqrt {3} ', 'rit_42100_3': ' \\frac {2 \\pm \\sqrt {2}} {4} ', 'rit_4275_2': ' \\frac {3} {4} ', 'rit_4235_0': ' \\frac {7} {5} ', 'rit_4230_3': ' \\frac {6} {3} ', 'rit_4235_2': ' 6 \\frac {\\sqrt {3}} {4} ', 'rit_

### From Predicted Labels collect all symbols also convert them to gt notation

In [25]:
def combine_all_predict(predict_labels):
    line = ""
    for i in range(predicted_labels.size):
        predicted = training_class_labels[predicted_labels[i]]
        line += convert_into_gt(predicted) + " "
        
    return line

def convert_into_gt(input_symbol):
    symbols =['(',')','+','-','0','1','2','3','4','5','6','7','8','9','=','div','times']
    
    if input_symbol == 'div':
        return "/div"
    if input_symbol == 'times':
        return "/times"
    return input_symbol
    ## in gt symbals div are /div and times are /times
    
combine = combine_all_predict(predicted_labels)

### Compare with ground truth

In [33]:
print(combine)
def compare_w_ground(path,combined_predicted):
    ground_truth = filtered_data_dictionary.get(path)
    return string_similarity(combined_predicted,ground_truth)

def string_similarity(str1, str2):
    # Remove white spaces from both strings
    str1 = str1.replace(" ", "")
    str2 = str2.replace(" ", "")
    
    # Calculate the length of the longer string
    max_length = max(len(str1), len(str2))
    
    # Initialize a variable to count the number of matching characters
    matching_count = 0
    
    # Compare the characters of both strings
    for char1, char2 in zip(str1, str2):
        if char1 == char2:
            matching_count += 1
    
    # Calculate the percentage of similarity
    similarity_percentage = (matching_count / max_length) * 100
    
    return similarity_percentage

0 6 8 


In [34]:
print(compare_w_ground('ez',combine))

0.0


### Save Into Pandas DataFrame